## Import Dependancies

In [5]:
#scraping tools
from urllib.error import HTTPError
from datetime import datetime
import json
from bs4 import BeautifulSoup as bs
import pandas as pd
import os
import numpy as np
import requests

# NLP tools
import pandas as pd
from collections import Counter
import re
from nltk.stem import WordNetLemmatizer, PorterStemmer, SnowballStemmer
from nltk.corpus import stopwords
import nltk
from nltk.tokenize import word_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer

#import API key
from keys import magic_key

### Get YouTube Transcript (Currently Broken)

Apparently you can't scrape the transcripts the old fashioned way. 

In [ ]:
# print("Drop in the link to the youtube video.")
# url = input()

# # https://www.youtube.com/watch?v=jCZKEnThrjI

In [ ]:
# print (f'Your url is {url}')
# with request.urlopen(url) as response:
#     resp = response.read()
#     soup = bs(resp, "html.parser")
#     print("Your youtube soup is ready!")

In [ ]:
# print(soup)

In [ ]:
# #pull in title and transcript from the soup
# lil_soup = soup.find_all("div", id_="segments-container")
# print(lil_soup)

### Youtube API

In [14]:
#function for getting Youtube comments with the Youtube API
def get_comments():
    
    #get the parameters url from the user and create a 
    video_url = input("Drop in the Youtube Link: ")
    print ('Got it! Thanks')
    video_id = video_url[32:] #extract the video id from the url
    # print(video_id)
    
    num_comments = input("How many comments do you what to explore? ")
    max_comments = re.sub('[^0-9]','', num_comments) #drop any letters
    print(f'{max_comments}, huh? Whatever you say boss.')
    # print(max_comments)
    
    # init API query
    base_url = f"https://www.googleapis.com/youtube/v3/commentThreads?key={magic_key}"
    params = f"&textFormat=plainText&part=snippet&videoId={video_id}&maxResults={max_comments}"
    
    #CALL API
    query_url = base_url + params
    response = requests.get(query_url).json()
    
    
    # LOOP THRU COMMENTS AND GET THAT YUMMY COMMENT TEXT
    # response['items'][10] # this is the format of a SINGLE COMM
    #raw_comment_list = response['items']

    num_comments = response['pageInfo']['totalResults'] # number of comments acutally returned from API
    counter = np.arange(0,num_comments) # create list to count thru
    print(f'We found {num_comments} comments on this video.')

    comments = []
    for c in counter:
        comment_text = response['items'][c]['snippet']['topLevelComment']['snippet']['textOriginal']
        #print(comment_text)
        #print('------------------')
        comments.append(comment_text)
        
    return comments

In [10]:
comments = get_comments()

Drop in the Youtube Link: https://www.youtube.com/watch?v=DfWQLbIHPPk
Got it! Thanks
How many comments do you what to explore? 10
10, huh? Whatever you say boss.
We found 10 comments on this video.


In [22]:
sia = SentimentIntensityAnalyzer()

#init lists
text = []
neg_score = []
neu_score = []
pos_score = []
compound_score = []

for comment in comments:
    siaOut = sia.polarity_scores(comment)
    
    text.append(comment)
    neg_score.append(siaOut['neg'])
    neu_score.append(siaOut['neu'])
    pos_score.append(siaOut['pos'])
    compound_score.append(siaOut['compound'])
    
    print(comment)
    print(siaOut)
    print('-------------')

Oh I’ve missed you! I’ve been worried because you haven’t posted 🙂 How are you holding up?
{'neg': 0.251, 'neu': 0.749, 'pos': 0.0, 'compound': -0.5707}
-------------
No, it isn’t.  (At least not anywhere that I’m aware of.)
{'neg': 0.18, 'neu': 0.82, 'pos': 0.0, 'compound': -0.296}
-------------
THE SHADEEE
love the shade towards this nuts, how they fell for this lies and alt right mummy is beyond my reason, also the comedic timing is so on point 😂😂😂
{'neg': 0.148, 'neu': 0.679, 'pos': 0.173, 'compound': 0.2558}
-------------
Was she/they completely stoned all of their lives or what???
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
-------------
Not unusual..they were also caught worshiping a Mr. Potato Head in their back yard
{'neg': 0.0, 'neu': 0.857, 'pos': 0.143, 'compound': 0.25}
-------------
I would love to know how they mummified her. I know that mummification is historically a very complex and thorough process. i cant wrap my head around a small group of laypeople prop

In [23]:
# create a dataframe of the comments and their sentiments
df = pd.DataFrame({'comment': text,
                   'neg_score': neg_score,
                   'neu_score': neu_score,
                   'pos_score': pos_score,
                   'compound_score': compound_score
                  })

df

,comment,neg_score,neu_score,pos_score,compound_score
0,Oh I’ve missed you! I’ve been worried because ...,0.251,0.749,0.000,-0.5707
1,"No, it isn’t. (At least not anywhere that I’m...",0.180,0.820,0.000,-0.2960
2,"THE SHADEEE\nlove the shade towards this nuts,...",0.148,0.679,0.173,0.2558
3,Was she/they completely stoned all of their li...,0.000,1.000,0.000,0.0000
4,Not unusual..they were also caught worshiping ...,0.000,0.857,0.143,0.2500
5,I would love to know how they mummified her. I...,0.000,0.900,0.100,0.6369
6,Prosecutors have dropped criminal charges agai...,0.133,0.627,0.241,0.6705
7,I'm trying to figure out what to say all I can...,0.103,0.722,0.175,0.3818
8,I'm trying to figure out what to say all I can...,0.109,0.706,0.185,0.3818
9,"""Did you folks wake up this morning-""\nMe: ""nope""",0.000,1.000,0.000,0.0000


### Tokenize Document

In [ ]:
file = "Resources/Corpus_Podcast"

In [ ]:
with open(file) as f:
    raw_text = f.readlines()

In [ ]:
def to_tokens(dirty_text):
    
    big_string = ' '.join(dirty_text).lower() # join and lower into big string
    big_string = re.sub(r"[^a-z0-9'\s]", '', big_string) #drop punctuation
    big_string = re.sub(r'\d+', '', big_string) # remove numbers
    
    tokens = nltk.word_tokenize(big_string) # tokenize with nltk
    print(f'Your text is now {len(tokens)} tokens.')
    
    print("Drop it like it's stop.")
    #loop thru words and strip stop words
    cleaned_words = []
    for token in tokens:
        if token not in stopwords.words('english'):
            cleaned_words.append(token)
            
    print(f'Now your text is only {len(cleaned_words)} tokens.')
    
    return tokens

In [ ]:
tokens = to_tokens(raw_text)
tokens

In [ ]:
def score_text(dirty_text):
    
    big_string = ' '.join(dirty_text).lower() # join and lower into big string
    big_string = big_string.replace('.','').replace(',','') #drop commas and periods
    
    sia = SentimentIntensityAnalyzer()
    siaOut = sia.polarity_scores(big_string)
    print(siaOut)

In [ ]:
score_text(tokens)

In [ ]:
score_text(raw_text)